In [ ]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

In [ ]:
#자사고 폐지에 대한 인식을 2022년도 언론보도를 통해 보고자 함.
#크롤링, 데이터 정제, 워드클라우드 순으로 봄.
#네이버를 활용하여 2022.01.01~2022.12.13까지 총 1950건이 수집됨.

In [ ]:
path = "chromedriver.exe"

driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.naver.com/")
time.sleep(2)

In [ ]:
query_text = input("검색어 입력: ")

element = driver.find_element(By.ID, "query");

element.send_keys(query_text)
element.submit()
time.sleep(2)

In [ ]:
driver.find_element(By.LINK_TEXT, "뉴스").click()
time.sleep(1)

driver.find_element(By.LINK_TEXT, "최신순").click()
time.sleep(1)

driver.find_element(By.LINK_TEXT, "옵션").click()
time.sleep(1)

In [ ]:
# 직접 입력 클릭
driver.find_element(By.CSS_SELECTOR, '#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a.txt.txt_option._calendar_select_trigger').click()
time.sleep(1)

# 기간 설정시작 클릭
driver.find_element(By.CSS_SELECTOR, "#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.set_calendar > span:nth-child(1) > a").click()
time.sleep(1)


# 월 - 1월 클릭
driver.find_element(By.CSS_SELECTOR, "#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(2) > div > div > div > ul > li:nth-child(1) > a").click()
time.sleep(1)

# 일 - 1일 클릭
driver.find_element(By.CSS_SELECTOR, "#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(3) > div > div > div > ul > li:nth-child(1) > a").click()
time.sleep(1)

# 기간 설정란 적용 버튼 클릭
driver.find_element(By.CSS_SELECTOR, "#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.btn_area > button").click()
time.sleep(1)

In [ ]:
# 뉴스 제목, url, 날짜, 신문사 가져와서 저장 후 마지막 페이지까지 반복
url_list = []
title_list = []
while True:
    # 검색 결과 뉴스 url과 제목 가져오기
    class_articles = ".news_tit"
    url_link = driver.find_elements(By.CSS_SELECTOR, class_articles)

    for article in url_link:
        url = article.get_attribute('href')
        url_list.append(url)

    for article in url_link:
        title = article.get_attribute('title')
        title_list.append(title)

    # 다음 페이지 버튼 클릭
    next_btn = driver.find_element(By.CSS_SELECTOR, "#main_pack > div.api_sc_page_wrap > div > a.btn_next")
    # 마지막 페이지일 경우 다음 페이지 버튼이 비활성화됨. 이것을 확인하는 코드
    next_btn_disabled = True if next_btn.get_attribute("aria-disabled") == 'true' else False
    
    next_btn.click()
    time.sleep(1)
    
    # 마지막 페이지일 경우 while 반복문 탈출
    if next_btn_disabled:
        break

In [ ]:
# 수집 결과 확인
title_list

In [ ]:
# url과 제목 csv 파일로 저장함
df = pd.DataFrame({'url': url_list, 'title':title_list})
df.to_csv("2022jasago_news_url_title.csv", index=False, encoding='UTF-8')
df.to_csv("2022jasago_news_url_title.txt", index=False, encoding='UTF-8')